In [61]:
import os, glob, chess.pgn, utils
import numpy as np
import tensorflow as tf

conf = utils.Config()

inputs = []
outputs = []

# https://database.nikonoel.fr downloaded from here

path = '/home/marcello/github/ChessBreaker/data/Database'
dataset_path = '/home/marcello/github/ChessBreaker/data/dataset'

def count(path):
    info = {}
    files = 0
    for filename in glob.glob(os.path.join(path, '*.pgn')):
        print(filename)
        files += 1

        with open(os.path.join(os.getcwd(), filename), 'r') as pgn:
            moves = 0
            games = 0
            game = chess.pgn.read_game(pgn)

            while game != None:
                games += 1

                whole_game_moves = game.game().mainline_moves()
                for move in whole_game_moves:
                    moves += 1
                
                game = chess.pgn.read_game(pgn)

            info[filename] = {}
            info[filename]["games"] = games
            info[filename]["total_moves"] = moves

    return info


small_file = "/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-11.pgn"
def gen():
    planes = None
    output_array = np.zeros([*conf.BOARD_SHAPE, conf.N_PLANES], dtype=conf.PLANES_DTYPE_NP)
    
    for filename in [small_file]:
    # for filename in glob.glob(os.path.join(database_path, '*.pgn')): # for whole dataset
        with open(os.path.join(os.getcwd(), filename), 'r') as pgn:
            game = chess.pgn.read_game(pgn)

            while game != None:
                whole_game_moves = game.game().mainline_moves()
                board = chess.Board()
                board_history = [board.fen()[:-6]]
                
                for move in whole_game_moves:
                    # the input is the PREVIOUS board
                    planes = utils.update_planes(planes, board, board_history)
                    # inputs.append(planes)
                    
                    # the output is the move from that position
                    mask = utils.mask_moves([move])[0]
                    output_array[mask[0], mask[1], mask[2]] = 1
                    # outputs.append(output_array)

                    yield (planes, output_array) ### yield before resetting the output

                    output_array[mask[0], mask[1], mask[2]] = 0
                    
                    # then you actually push the move preparing for next turn
                    board.push(move)
                    board_history.append(board.fen()[:-6])
                
                game = chess.pgn.read_game(pgn)

# avg = np.average([info[file, "games"] for file in info.keys()])

In [62]:
# info = count(path)

In [63]:
%cd "data/"

[Errno 2] No such file or directory: 'data/'
/home/marcello/github/ChessBreaker/data


In [64]:
import json
# with open("info.json", "w") as f:
#     json.dump(info, f)

with open("info.json", "r") as f:
    info = json.load(f)

In [65]:
total_games = 0
for filename in info:
    total_games += info[filename]["games"]
total_games

3819130

In [66]:
total_moves = 0
for filename in info:
    total_moves += info[filename]["total_moves"]
total_moves

313831972

In [67]:
import pandas as pd

df = pd.DataFrame(info)
df = df.transpose()
df = df.sort_values(by=['games'], ascending=True)
df.head(50)

,games,total_moves
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-01.pgn,3,299
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2013-11.pgn,4,409
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-03.pgn,5,457
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-04.pgn,7,585
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2013-09.pgn,10,591
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-02.pgn,23,1626
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-07.pgn,92,8379
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-08.pgn,138,11505
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-06.pgn,140,13136
/home/marcello/github/ChessBreaker/data/Database/lichess_elite_2014-09.pgn,207,15587


In [68]:
dataset = tf.data.Dataset.from_generator(gen,
    output_signature=(
    tf.TensorSpec((8,8,119), dtype=tf.dtypes.float16),
    tf.TensorSpec((8,8,73), dtype=tf.dtypes.float16)
))

tf.data.experimental.save(dataset, dataset_path)